# Go through each ISSN-L and either update in OpenAlex, add to OpenAlex, or flag for inspection

In [1]:
import sys

In [2]:
sys.path.append('../..')

In [3]:
from datetime import datetime, timezone
import json
from tqdm import tqdm

In [4]:
import pandas as pd

In [5]:
from cleanup.util import entities_by_ids, paginate_openalex, make_request

In [30]:
from app import db
from models import Source
from sqlalchemy import text
from sqlalchemy.orm import Load
from sqlalchemy.exc import MultipleResultsFound

In [7]:
%%time
url = "https://api.openalex.org/sources"
params = {
    'mailto': 'jportenoy@ourresearch.org',
    'group_by': 'issn',
}
data = []
for r in paginate_openalex(url, params=params):
    data.extend(r.json()['group_by'])

CPU times: user 7.95 s, sys: 369 ms, total: 8.32 s
Wall time: 2min 33s


In [8]:
df_sources = pd.DataFrame(data)
_rename = {
    'key': 'issn',
    'count': 'num_sources_in_openalex',
}
df_openalex_issn_sources_count = df_sources.rename(columns=_rename).drop(columns=['key_display_name'])

In [9]:
df_openalex_issn_sources_count['num_sources_in_openalex'].value_counts()

num_sources_in_openalex
1    201232
2      7008
3       140
4         6
5         2
Name: count, dtype: int64

In [10]:
path_to_simple_file = '../data/issn_audit_20240301/ISSNs_openalex.xlsx'
df_issn_ic = pd.read_excel(path_to_simple_file, sheet_name='Sheet')

In [11]:
columns_rename = {
    colname: colname.lower().replace(' ', '_')
    for colname in df_issn_ic.columns
}
df_issn_ic.rename(columns=columns_rename, inplace=True)

In [12]:
df_issn_ic.dropna(subset=['issn'], inplace=True)

In [13]:
# drop duplicate rows, to get one ISSN per row, as reported by ISSN-IC
df_issn_ic['comment'] = df_issn_ic['comment'].fillna(value='')
df_issn_ic = df_issn_ic.sort_values(['issn', 'comment']).drop_duplicates(subset='issn', keep='last')

In [14]:
df_issn_ic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214606 entries, 110805 to 118679
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   submitted_1348-0278  214606 non-null  object
 1   issn                 214606 non-null  object
 2   issnl                213508 non-null  object
 3   category             214606 non-null  object
 4   medium               213309 non-null  object
 5   frequency            214583 non-null  object
 6   country              214606 non-null  object
 7   center               214606 non-null  object
 8   language             213239 non-null  object
 9   start                214606 non-null  object
 10  end                  214606 non-null  object
 11  title                213622 non-null  object
 12  comment              214606 non-null  object
dtypes: object(13)
memory usage: 22.9+ MB


In [15]:
# get dict of issnl to issn
x = df_issn_ic.dropna(subset=['issnl'])
x = x[x['category'].isin(['Register', 'Work', 'Free', 'Validation Request'])]
x = x[['issn', 'issnl']]
issnl_to_issn = {}
ignore = [
    '0000-0000',
]
x = x[~(x['issnl'].isin(ignore))]
for name, gbdf in x.groupby('issnl'):
    if name in ignore:
        continue
    issnl_to_issn[name] = gbdf['issn'].tolist()
len(issnl_to_issn)

140984

In [16]:
%%time
# get one title per issnl
issnl_display_name = {}
lookup = df_issn_ic.set_index('issn')['title']
for issnl in issnl_to_issn:
    try:
        issnl_display_name[issnl] = lookup.loc[issnl]
    except KeyError:
        issnl_display_name[issnl] = df_issn_ic[df_issn_ic['issnl']==issnl].iloc[0]['title']

CPU times: user 11.5 s, sys: 20.4 ms, total: 11.5 s
Wall time: 11.5 s


In [17]:
issn_sources_count = df_openalex_issn_sources_count.set_index('issn')['num_sources_in_openalex'].astype(int)

In [24]:
def insert_notfound(issnl, session, recorded_at=None):
    if recorded_at is None:
        recorded_at = datetime.now(timezone.utc).isoformat()
    sq = """INSERT INTO issn_audit_20240321.update1_issnl_notfound
        (issnl, recorded_at)
        VALUES(:issnl, :recorded_at);"""
    session.execute(text(sq), {
        'issnl': issnl,
        'recorded_at': recorded_at,
    }) 

In [25]:
def insert_multfound(issnl, num_found, session, recorded_at=None):
    if recorded_at is None:
        recorded_at = datetime.now(timezone.utc).isoformat()
    sq = """INSERT INTO issn_audit_20240321.update1_issnl_multfound
        (issnl, num_found, recorded_at)
        VALUES(:issnl, :num_found, :recorded_at);"""
    session.execute(text(sq), {
        'issnl': issnl,
        'num_found': num_found,
        'recorded_at': recorded_at,
    }) 

In [26]:
def get_source_from_db(issn, session) -> Source:
    source = session.query(Source).options(Load(Source).lazyload('*')).filter_by(issn=issn).one_or_none()
    if source is None:
        source = session.query(Source).options(Load(Source).lazyload('*')).filter(Source.issns.contains(issn)).one()
    return source


In [27]:
def update_source(issnl, issn_list, title, source_type='journal', updated_date=None, session=None):
    if session is None:
        session = db.session
    if updated_date is None:
        updated_date = datetime.now(timezone.utc).isoformat()
    source = get_source_from_db(issnl, session)
    if source is None:
        raise KeyError(f"issnl {issnl} not found in db")
    old_issnl = source.issn
    old_display_name = source.display_name
    old_issns = source.issns
    old_type = source.type

    # make updates
    if title:
        source.display_name = title
    source.issn = issnl
    source.issns = json.dumps(issn_list)
    source.issns_text_array = issn_list
    source.type = source_type
    source.updated_date = updated_date
    session.add(source)

    # insert into log table
    sq = """INSERT INTO issn_audit_20240321.update1
        (issnl, old_issns, new_issns, old_issnl, old_display_name, new_display_name, updated_date, source_id, old_type, new_type)
        VALUES(:issnl, :old_issns, :new_issns, :old_issnl, :old_display_name, :new_display_name, :updated_date, :source_id, :old_type, :new_type);
        """
    session.execute(text(sq), {
        'issnl': issnl,
        'old_issns': old_issns,
        'new_issns': json.dumps(issn_list),
        'old_issnl': old_issnl,
        'old_display_name': old_display_name,
        'new_display_name': title if title else old_display_name,
        'updated_date': updated_date,
        'source_id': source.id,
        'old_type': old_type,
        'new_type': source_type,
    }) 

In [32]:
try:
    now = datetime.now(timezone.utc).isoformat()
    # for issnl, issn_list in issnl_to_issn.items():
    for issnl, issn_list in tqdm(issnl_to_issn.items(), total=len(issnl_to_issn)):
        title = issnl_display_name[issnl]
        num_found = int(issn_sources_count.get(issnl, 0))
        if num_found > 1:
            insert_multfound(issnl, num_found, db.session, recorded_at=now)
        elif num_found == 0:
            insert_notfound(issnl, db.session, recorded_at=now)
        else:
            try:
                if issnl not in issn_list:
                    issn_list.append(issnl)
                # move issnl to the front
                issn_list.insert(0, issn_list.pop(issn_list.index(issnl)))
                update_source(issnl, issn_list, title, updated_date=now, session=db.session)
            except MultipleResultsFound:
                print(f"multiple results found: {issnl}")
                insert_multfound(issnl, -1, db.session, recorded_at=now)

    db.session.commit()
finally:
    db.session.close()

  0%|          | 73/140984 [00:02<1:24:51, 27.68it/s]

multiple results found: 0001-5113


  1%|          | 1219/140984 [00:46<1:34:20, 24.69it/s]

multiple results found: 0006-3835


  1%|          | 1322/140984 [00:50<1:29:40, 25.96it/s]

...

 90%|█████████ | 127316/140984 [1:25:52<09:37, 23.66it/s]

multiple results found: 2752-7646


 93%|█████████▎| 131470/140984 [1:29:07<07:57, 19.93it/s]

multiple results found: 2792-6818


100%|██████████| 140984/140984 [1:37:18<00:00, 24.15it/s]
